# import modules

In [1]:
import pandas as pd
import numpy as np 
import datetime
import time
import os 
from platform import python_version

In [2]:
print(f'python version: {python_version()}')
print(f'pandas version : {pd.__version__}')
print(f'numpy version : {np.__version__}')

python version: 3.7.4
pandas version : 1.1.1
numpy version : 1.18.5


# Data set 불러오기

In [3]:
# 저장한 디렉토리 저장
directory = os.getcwd()

In [4]:
X_df=pd.read_excel(directory+'/2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx',header = 1) 

In [5]:
X = X_df.copy()

In [6]:
X.head() 

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN


In [7]:
# 취급액을 예측해야 하므로 모두 결측치 처리가 되어있다. 이에 해당 열을 제외했다.
X.drop(columns='취급액',inplace= True)

# 결측치 확인 및 EDA

In [8]:
X.isnull().sum()

방송일시        0
노출(분)    1111
마더코드        0
상품코드        0
상품명         0
상품군         0
판매단가        0
dtype: int64

## 노출(분) 결측치 처리

In [9]:
#노출분은 같은 시간에 같이 방송되어서 NaN 으로 뜨는것이다.
#같은 시간에 방송되어진 값을 평균해서 채운다.

In [10]:
X.loc[:,'평균방송분'] = X.loc[:,'노출(분)']

In [11]:
# 0 노출시간이 0 인 데이터가 없다.
# 즉 Na 처리할떄 0으로 대체한 후에 처리하였다.
(X.loc[:,'노출(분)']==0).sum()

0

In [12]:
# 노출(분) 을 제일 먼저 시작한 시간의 값으로 만드는 FUNCTION 정의
def NA_minute(df=X):
    # 곁측치를 0으로 대체
    df['노출(분)'] = df['노출(분)'].fillna(0)
    
    count_0 = [] 
    count_not0 = [] # 0이 아닌 수의 index
    for idx in df.index :
        if df.loc[idx,'노출(분)'] == 0 :
            count_0.append(idx)
        else :
            count_not0.append(idx)
    count_not0.append(len(df)) # 맨 마지막 열에 0 인 경우를 대비하여

    for idx,x in enumerate(count_not0):
        if idx != len(count_not0)-1: 
            if (count_not0[idx+1] - count_not0[idx] - 1) != 0:
                start = count_not0[idx] 
                end = count_not0[idx+1]
                num = end - start 
                df.loc[start:(end-1),'노출(분)'] = df.loc[start,'노출(분)'] # / num 을 붙여주면 평균으로 만들게 됩니다.
    return df

In [13]:
X = NA_minute(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000
...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000


## 얼마나 같은 방송일시에 겹쳐있는지 변수늘리기

In [14]:
# 겹치는 수는 같은 편성을 연속해서 했을때의 경우를 고려한 것이다.
# 같은 편성을 했다는 기준은 마더코드를 기준으로 삼았다.(즉 마더코드가 같은 방송이 연속되면, 겹쳤다고 판단.)
# 예를 들어 삼성 tv 를 4시/4시20분/4시40분 3번 편성했다면 겹치는수는 3이 된다.

In [15]:
X.loc[:,'겹치는수'] = 1

In [16]:
for idx in range(1,len(X))  :
    if ~(X.loc[idx-1,'마더코드'] == X.loc[idx,'마더코드']): 
        continue
    elif (X.loc[idx-1,'마더코드'] == X.loc[idx,'마더코드']) : 
        if X.loc[idx-1,'방송일시'] == X.loc[idx,'방송일시']: 
            X.loc[idx,'겹치는수'] = X.loc[idx-1,'겹치는수']
        elif ~(X.loc[idx-1,'방송일시'] == X.loc[idx,'방송일시']): 
            X.loc[idx,'겹치는수'] = X.loc[idx-1,'겹치는수']+1

In [17]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2
...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,1
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,2
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,3
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1


## 평균 방송분 변수늘리기

In [18]:
# 각 방송에대한 평균 방송분 열을 추가하였다.
# 즉 6:00 ~ 6:20 분동안 4개의 상품이 방송되었다면, 평균적으로 5분 노출한것이므로, 평균방송분은 5분이 된다.

In [19]:
def MEAN_minute(df=X):
    # 곁측치를 0으로 대체
    df['평균방송분'] = df['평균방송분'].fillna(0)
    
    count_0 = [] 
    count_not0 = [] # 0이 아닌 수의 index
    for idx in df.index :
        if df.loc[idx,'평균방송분'] == 0 :
            count_0.append(idx)
        else :
            count_not0.append(idx)
    count_not0.append(len(df)) # 맨 마지막 열에 0 인 경우를 대비하여

    for idx,x in enumerate(count_not0):
        if idx != len(count_not0)-1: 
            if (count_not0[idx+1] - count_not0[idx] - 1) != 0:
                start = count_not0[idx] 
                end = count_not0[idx+1]
                num = end - start 
                df.loc[start:(end-1),'평균방송분'] = df.loc[start,'평균방송분'] / num
    return df

In [20]:
X = MEAN_minute(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2
...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,1
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,2
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,3
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1


## 시간 데이터 늘리기

In [21]:
# 방송일시를 쪼개서 시간데이터를 최대로 늘렸다.
# 년,월,알,시간,시분 등으로 나누었고, 이는 뒤에 변수를 조함해 추가할떄 유용하게 쓰이지만, 직접적인 수치형 데이터로 쓰이는 열은 많지 않다.

In [22]:
def time_feature(df = X):
    df['방송일시']=df['방송일시'].astype('str')
    df['날짜']=df['방송일시'].str.split(' ').str[0]
    df['시간']=df['방송일시'].str.split(' ').str[1]
    df['년']=df['날짜'].str.split('-').str[0]
    df['월']=df['날짜'].str.split('-').str[1]
    df['일']=df['날짜'].str.split('-').str[2]
    df['시']=df['시간'].str.split(':').str[0]
    df['분']=df['시간'].str.split(':').str[1]
    df['초']=df['시간'].str.split(':').str[2]
    df['시각'] = df['시'].astype('float32') + df['분'].astype('float32')*(1/60)
    df['시분'] = df['시'] + ':' + df['분']
    df.drop(['시','분','초','시간'],axis=1,inplace = True)
    return(df)

In [23]:
X = time_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,년,월,일,시각,시분
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,2020,06,01,6.333333,06:20
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,2020,06,01,6.666667,06:40
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,2020,06,01,7.000000,07:00
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,2020,06,01,7.333333,07:20
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,2020,06,01,7.666667,07:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,1,2020-07-01,2020,07,01,0.333333,00:20
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,2,2020-07-01,2020,07,01,0.666667,00:40
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,3,2020-07-01,2020,07,01,1.000000,01:00
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,2020,07,01,1.333333,01:20


## 시청률 평균 적용

In [24]:
# 주최측에서 제공한 시청률 데이터를 이용하여 주중과 주말(금요일 포함) 의 시청률에 대한 평균을 생성하여 별도의 엑셀 파일로 저장했다. 
# 이를 불러와서 test set 분석에 활용했다.

In [25]:
weekend_df=pd.read_csv(directory+'/weekend.csv')
weekday_df=pd.read_csv(directory+'/weekday.csv')

In [26]:
weekend=weekend_df.copy()
weekday=weekday_df.copy()
weekend.index = weekend['시각']
weekday.index = weekday['시각']

In [27]:
# 2020년 6월 금요일, 주말 및 공휴일의 날짜
time_list=['0605','0606','0607','0612','0613','0614','0619','0620','0621','0626','0627','0628']

In [28]:
# 금토일,공휴일 / 월화수목 의 시청률 평균을 다르게 계산하였습니다.
# 방송지속시간 동안의 이전셀에서 계산한 시청률 평균을 모두 더한 후, 방송 지속시간으로 평균을 낸 것이다.

In [29]:
X['월일'] = X['월'].astype(str) + X['일'].astype(str)
X.loc[:,'시분'] = X.loc[:,'시분'].astype(str)
X['시간열']=X['시분'].str.split(':').str[0].astype(str) + X['시분'].str.split(':').str[1].astype(str)
X['시간열']=X['시간열'].astype(int)

In [30]:
for i in range(len(X)):
    if X.loc[i,'시간열'] < 600:
         X.loc[i,'시간열']  += 2400

In [31]:
X['정수노출(분)']=X['노출(분)'].round(0).astype(int)

In [32]:
X['시청률평균'] = 0

In [33]:
for idx in range(len(X)-1):
    if X.loc[idx,'월일'] in time_list:
        if X.loc[idx,'시간열'] == X.loc[idx+1,'시간열']: 
            X.loc[idx,'시청률평균'] = 1
        elif X.loc[idx,'시간열'] > X.loc[idx+1,'시간열'] : 
             X.loc[idx,'시청률평균'] = np.mean(weekend.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)'])-1,'시청률 평균'])
        elif X.loc[idx,'시간열'] < X.loc[idx+1,'시간열'] :
             X.loc[idx,'시청률평균'] = np.mean(weekend.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)'])-1,'시청률 평균'])
    else:
        if X.loc[idx,'시간열'] == X.loc[idx+1,'시간열']: 
            X.loc[idx,'시청률평균'] = 1
        elif X.loc[idx,'시간열'] > X.loc[idx+1,'시간열'] : 
             X.loc[idx,'시청률평균'] = np.mean(weekday.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)'])-1,'시청률 평균'])
        elif X.loc[idx,'시간열'] < X.loc[idx+1,'시간열'] : 
             X.loc[idx,'시청률평균'] = np.mean(weekday.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)'])-1,'시청률 평균'])

In [34]:
idx = len(X)-1
if X.loc[idx,'월일'] in time_list:
    X.loc[idx,'시청률평균'] = np.mean(weekend.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)'])-1,'시청률 평균'])
else:
    X.loc[idx,'시청률평균'] = np.mean(weekday.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)'])-1,'시청률 평균'])

In [35]:
for idx in range(len(X)-1,-1,-1): 
    if X.loc[idx,'시청률평균'] == 1: 
        X.loc[idx,'시청률평균'] = X.loc[idx+1,'시청률평균'] 

In [36]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,년,월,일,시각,시분,월일,시간열,정수노출(분),시청률평균
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,2020,06,01,6.333333,06:20,0601,620,20,0.001522
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,2020,06,01,6.666667,06:40,0601,640,20,0.001564
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,2020,06,01,7.000000,07:00,0601,700,20,0.001550
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,2020,06,01,7.333333,07:20,0601,720,20,0.002319
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,2020,06,01,7.666667,07:40,0601,740,20,0.003285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,1,2020-07-01,2020,07,01,0.333333,00:20,0701,2420,20,0.005520
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,2,2020-07-01,2020,07,01,0.666667,00:40,0701,2440,20,0.005907
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,3,2020-07-01,2020,07,01,1.000000,01:00,0701,2500,20,0.005375
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,2020,07,01,1.333333,01:20,0701,2520,20,0.004828


## 예측할 index 뽑아내기

In [37]:
# 예측할 INDEX 를 뽑아내기 위해 먼저 index 번호를 저장한다.
X['index'] =X.index

## 상품군 결측치 처리

In [38]:
 # 판매단가가 0 인 상품은 예측에서 제외

In [39]:
def NA_goods(df=X):
    df=df.loc[df['판매단가'] != 0,:]
    df = df.reset_index(drop=True)
    return df

In [40]:
X = NA_goods(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,년,월,일,시각,시분,월일,시간열,정수노출(분),시청률평균,index
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,2020,06,01,6.333333,06:20,0601,620,20,0.001522,0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,2020,06,01,6.666667,06:40,0601,640,20,0.001564,1
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,2020,06,01,7.000000,07:00,0601,700,20,0.001550,2
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,2020,06,01,7.333333,07:20,0601,720,20,0.002319,3
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,2020,06,01,7.666667,07:40,0601,740,20,0.003285,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,2020,07,01,0.166667,00:10,0701,2410,10,0.005167,2883
2712,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,2020,07,01,0.166667,00:10,0701,2410,10,0.005167,2884
2713,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,2020,07,01,0.166667,00:10,0701,2410,10,0.005167,2885
2714,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,2020,07,01,1.333333,01:20,0701,2520,20,0.004828,2889


##  토요일 결측치 처리

매주 토요일 18:00~18:20은 정보방송시간으로 추정 제외(당사사정에 따라 ±20 편성 변경 있음)

In [41]:
def NA_Saturday(df = X):
     # 년, 월, 일을 대입하면 요일을 반환하는 함수
    t = ['월','화','수','목','금','토','일']
    r = datetime.datetime.today()
    def printDayOfTheWeek(year, month, day):
        dayOfTheWeek = ['월','화','수','목','금','토','일']
        return dayOfTheWeek[datetime.datetime(year,month,day).weekday()]

    day_list = []
    # 방송일시 열에서 년, 월, 일, 요일, 시간을 뽑아내기
    for date in X.iloc[:,0]:
        date = str(date)
        year = int(date[:4])
        month = int(date[5:7])
        day = int(date[8:10])
        first = printDayOfTheWeek(year,month,day)
        second = date[11:16]
        # 각 행의 요일과 시간을 하나의 문자열로 만들어 day_list 리스트에 저장
        day_list.append(first+second)
    # train set에 요일&시간 열 추가
    X["요일/시간"] = day_list
    # 토요일 18시 0~19분에 시작하는 모든 방송 제거하기
    for i in range(len(df)):
        time = df.loc[i,'요일/시간']
        if (time[0:3]=="토18" and int(time[4:])<20):
            df = df.drop(i)
    return df

In [42]:
for i in range(len(NA_Saturday(X))):
    time = X.loc[i,'요일/시간']
    if time[0:3]=="토18":
        print(time)
# 하루 존재

토18:20
토18:40
토18:20
토18:20
토18:40
토18:40
토18:00
토18:40


In [43]:
X=X.reset_index(drop=True)

In [44]:
X = NA_Saturday(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,월,일,시각,시분,월일,시간열,정수노출(분),시청률평균,index,요일/시간
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,06,01,6.333333,06:20,0601,620,20,0.001522,0,월06:20
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,06,01,6.666667,06:40,0601,640,20,0.001564,1,월06:40
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,06,01,7.000000,07:00,0601,700,20,0.001550,2,월07:00
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,06,01,7.333333,07:20,0601,720,20,0.002319,3,월07:20
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,06,01,7.666667,07:40,0601,740,20,0.003285,4,월07:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,07,01,0.166667,00:10,0701,2410,10,0.005167,2883,수00:10
2712,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,07,01,0.166667,00:10,0701,2410,10,0.005167,2884,수00:10
2713,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,07,01,0.166667,00:10,0701,2410,10,0.005167,2885,수00:10
2714,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,07,01,1.333333,01:20,0701,2520,20,0.004828,2889,수01:20


## 예측할 인덱스 excel 저장

In [45]:
# 결측치를 DROP 하는 부분이 끝났으므로, 이제 이 index 를 저장합니다.

In [46]:
predict_index = X[['index']]

In [47]:
predict_index.to_csv('predict_index.csv')

In [48]:
X.drop(columns=['index'],inplace=True)

## 시간 변수 세분화

특정 시간대에 홈쇼핑 방송 시청이 활발하거나, 홈쇼핑 상품 매출이 높을 것이라 생각하여 2시간으로 세분화한 시간 변수를 사용했다.

In [49]:
def time_2h(df = X):
    # 2시간으로 세분화한 시간 변수 열을 추가한다. 우선 0을 원소로 넣는다.
    df['06:00~08:00'] = 0
    df['08:00~10:00'] = 0
    df['10:00~12:00'] = 0
    df['12:00~14:00'] = 0
    df['14:00~16:00'] = 0
    df['16:00~18:00'] = 0
    df['18:00~20:00'] = 0
    df['20:00~22:00'] = 0
    df['22:00~00:00'] = 0
    df['00:00~02:00'] = 0
    df['02:00~04:00'] = 0
    
    # 대소 비교가 용이한 '시간열'변수를 활용해 세분화한 시간대에 따라 1을 채워 넣는다.
    df.loc[(600<=df['시간열'])&(df['시간열']<800),'06:00~08:00'] = 1
    df.loc[(800<=df['시간열'])&(df['시간열']<1000),'08:00~10:00'] = 1
    df.loc[(1000<=df['시간열'])&(df['시간열']<1200),'10:00~12:00'] = 1
    df.loc[(1200<=df['시간열'])&(df['시간열']<1400),'12:00~14:00'] = 1
    df.loc[(1400<=df['시간열'])&(df['시간열']<1600),'14:00~16:00'] = 1
    df.loc[(1600<=df['시간열'])&(df['시간열']<1800),'16:00~18:00'] = 1
    df.loc[(1800<=df['시간열'])&(df['시간열']<2000),'18:00~20:00'] = 1
    df.loc[(2000<=df['시간열'])&(df['시간열']<2200),'20:00~22:00'] = 1
    df.loc[(2200<=df['시간열'])&(df['시간열']<2400),'22:00~00:00'] = 1
    df.loc[(2400<=df['시간열'])&(df['시간열']<2600),'00:00~02:00'] = 1
    df.loc[(2600<=df['시간열'])&(df['시간열']<2800),'02:00~04:00'] = 1
    return(df)

In [50]:
X = time_2h(X)

## 요일별 세분화

요일에 따라 홈쇼핑 방송 시청이나 홈쇼핑 상품 매출에 유의미한 차이가 있을 것이라 생각하여 요일에 따라 세분화한 시간 변수를 추가했다.

In [51]:
def dayoftheweek(df = X):
    dayday = []
    # 기존의 '요일/시간' 변수에서 구한 요일만 따로 추출하여 범주형 변수를 만든다.
    for day in X.loc[:,'요일/시간']:
        onlyday = str(day[:1])
        dayday.append(onlyday)
            
    X["요일"] = dayday
    return df

In [52]:
X = dayoftheweek(X)

## 분기별 세분화

2020년 6월은 모두 2분기에 해당하므로 '분기'열에 '2분기' 값을 넣었다.

In [53]:
# 분기를 train 하고 똑같이 처리.
X['분기']="2분기" 

In [54]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,12:00~14:00,14:00~16:00,16:00~18:00,18:00~20:00,20:00~22:00,22:00~00:00,00:00~02:00,02:00~04:00,요일,분기
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,0,0,0,0,0,0,0,0,월,2분기
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,0,0,0,0,0,0,0,0,월,2분기
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,0,0,0,0,0,0,0,0,월,2분기
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,0,0,0,0,0,0,0,0,월,2분기
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,0,0,0,0,0,0,0,0,월,2분기
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,0,0,0,0,0,0,1,0,수,2분기
2712,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,0,0,0,0,0,0,1,0,수,2분기
2713,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,0,0,0,0,0,0,1,0,수,2분기
2714,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,0,0,0,0,0,0,1,0,수,2분기


## 강수량 변수

강수량 데이터의 경우 기상청의 기상자료개방포털의 공공데이터를 사용했다.(https://data.kma.go.kr/stcs/grnd/grndRnList.do?pgmNo=69) 강수량이 높으면 집에서 활동하는 인구가 늘어 홈쇼핑 매출에 큰 영향이 있을 것이라 생각했고, 이에 일별 강수량을 고려했다.

In [55]:
# 외부데이터 불러오기
# 이 경우에도 외부데이터의 이름은 20rain.csv 의 이름으로 저장하였다.
rain_df=pd.read_csv(directory+'/20rain.csv',header = 7, encoding='CP949')
rain = rain_df.copy()

In [56]:
rain.head()

,날짜,지점,강수량(mm)
0,2020-06-01,전국,0.3
1,2020-06-02,전국,0.5
2,2020-06-03,전국,0.2
3,2020-06-04,전국,0.0
4,2020-06-05,전국,0.0


In [57]:
# 날짜에 맞게 강수량 데이터를 병합 
# 병합하며 7월 데이터 8 row 제거
def rain_feature(df = X):
       
    df = pd.merge(df, rain, on="날짜", how= "inner")
    df.drop(['지점'],axis=1,inplace = True)
    return(df)

X = rain_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,14:00~16:00,16:00~18:00,18:00~20:00,20:00~22:00,22:00~00:00,00:00~02:00,02:00~04:00,요일,분기,강수량(mm)
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,0,0,0,0,0,0,0,월,2분기,0.3
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,0,0,0,0,0,0,0,월,2분기,0.3
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,0,0,0,0,0,0,0,월,2분기,0.3
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,0,0,0,0,0,0,0,월,2분기,0.3
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,0,0,0,0,0,0,0,월,2분기,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,0,0,0,0,0,1,0,수,2분기,1.1
2711,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,0,0,0,0,0,1,0,수,2분기,1.1
2712,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,0,0,0,0,0,1,0,수,2분기,1.1
2713,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,0,0,0,0,0,1,0,수,2분기,1.1


## 온도 변수

기온 데이터 또한 마찬가지로 기상청의 기상자료개방포털의 공공데이터를 사용했다.(https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70) 기온에 따라 특정 상품군의 매출이 달라질 것이라 생각하여 일별 기온(평균기온, 최고기온, 최저기온)을 고려했다.

In [58]:
# 외부데이터 불러오기
# 기온 자료 또한 20temp.csv 의 이름으로 저장한 뒤 불러온다.
temp_df=pd.read_csv(directory+'/20temp.csv',header = 7, encoding='CP949')
temp = temp_df.copy()

In [59]:
temp.head()

,날짜,지점,평균기온(℃),최저기온(℃),최고기온(℃)
0,2020-06-01,전국,20.8,15.8,26.0
1,2020-06-02,전국,19.5,13.4,25.4
2,2020-06-03,전국,22.0,16.9,27.5
3,2020-06-04,전국,24.0,18.8,29.9
4,2020-06-05,전국,23.1,18.6,29.3


In [60]:
# 날짜에 맞게 기온 데이터를 병합
def temp_feature(df = X):
    df = pd.merge(df, temp, on="날짜", how= "inner")
    df.drop(['지점'],axis=1,inplace = True)
    return(df)
X = temp_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,20:00~22:00,22:00~00:00,00:00~02:00,02:00~04:00,요일,분기,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃)
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,0,0,0,0,월,2분기,0.3,20.8,15.8,26.0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,0,0,0,0,월,2분기,0.3,20.8,15.8,26.0
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,0,0,0,0,월,2분기,0.3,20.8,15.8,26.0
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,0,0,0,0,월,2분기,0.3,20.8,15.8,26.0
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,0,0,0,0,월,2분기,0.3,20.8,15.8,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,0,0,1,0,수,2분기,1.1,20.8,17.6,25.3
2711,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,0,0,1,0,수,2분기,1.1,20.8,17.6,25.3
2712,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,0,0,1,0,수,2분기,1.1,20.8,17.6,25.3
2713,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,0,0,1,0,수,2분기,1.1,20.8,17.6,25.3


## 연예인 변수 

상품명에 연예인 이름이 포함되는 경우 상품에 대한 평판과 신뢰도가 상승하여 매출에 긍정적인 효과가 있을 것으로 판단하였다. 연예인 변수를 이진변수로 추가하였다.

In [61]:
#연예인 이름이 포함되는 상품은 "연예인" 변수에서 1의 값을, 포함되지 않는 상품은 0의 값을 가진다. 
# train set 에 없지만, test set 에 있는 연예인을 추가하였습니다. 
def celebrity(df=X):
    df['연예인']=0
    list_celebrity=["전지현", "팽현숙", "전철우", "강레오", "김선영", "김정배",
                    "김규흔", "이봉원", "오세득", "유귀열", "이경제", "이만기",
                    "이보은", "임성근", "전철우", "최인선", "김병만", "김병지", 
                    "이동수", "서장훈", "송도순", "효재", "천수봉", 
                    "김정문", "숀리", "이정섭", "황성주","이숙","임화자"]
    for i in list_celebrity:
        df.loc[df.상품명.str.contains(i),'연예인']=1
    return df

In [62]:
X=celebrity(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,22:00~00:00,00:00~02:00,02:00~04:00,요일,분기,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,0,0,0,월,2분기,0.3,20.8,15.8,26.0,0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,0,0,0,월,2분기,0.3,20.8,15.8,26.0,0
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,0,0,0,월,2분기,0.3,20.8,15.8,26.0,0
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,0,0,0,월,2분기,0.3,20.8,15.8,26.0,0
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,0,0,0,월,2분기,0.3,20.8,15.8,26.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,0,1,0,수,2분기,1.1,20.8,17.6,25.3,0
2711,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,0,1,0,수,2분기,1.1,20.8,17.6,25.3,0
2712,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,0,1,0,수,2분기,1.1,20.8,17.6,25.3,0
2713,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,0,1,0,수,2분기,1.1,20.8,17.6,25.3,0


## 월급날

월급날에는 홈쇼핑 고객이 소비지출을 더 많이 하여 홈쇼핑 매출에도 긍정적인 영향을 줄 것으로 생각했으며 월급수령으로 인한 매출의 효과를 고려하기 위해 "월급날" 변수를 추가하였다. 고객 별로 월급 지급날이 상이하기 때문에 국민은행 공식 블로그를 참고하여 월급날을 10일, 20일, 25일로 정했다. 대체로 10일은 중소기업, 20일은 일부 공무원, 25일은 대기업의 월급 날이다. 월급 수령 후 소비지출의 증가 효과는 수령 당일 이후 며칠동안 더 지속될 것으로 생각하여 각 월급날의 +3일까지 "월급날" 변수에 포함했다.

In [63]:
#https://news.sbs.co.kr/news/endPage.do?news_id=N1005505452
#https://m.blog.naver.com/youngkbblog/221714986107

In [64]:
#10일~13일, 20~23일, 25일~28일에 판매한 상품은 "월급날" 변수에서 1의 값을, 그 이외 날은 0의 값을 갖는다. 
def income(df = X):
    df['월급날'] = 0
    df.loc[ ((10<=df['일'].astype('float32'))&(df['일'].astype('float32')<=13)) | ((20<=df['일'].astype('float32'))&(df['일'].astype('float32')<= 23)) | ((25<=df['일'].astype('float32')) & (df['일'].astype('float32')<= 28)),'월급날']  = 1
    return(df)

In [65]:
X=income(X);X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,00:00~02:00,02:00~04:00,요일,분기,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,0,0,월,2분기,0.3,20.8,15.8,26.0,0,0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,0,0,월,2분기,0.3,20.8,15.8,26.0,0,0
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,0,0,월,2분기,0.3,20.8,15.8,26.0,0,0
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,0,0,월,2분기,0.3,20.8,15.8,26.0,0,0
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,0,0,월,2분기,0.3,20.8,15.8,26.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,1,0,수,2분기,1.1,20.8,17.6,25.3,0,0
2711,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,1,0,수,2분기,1.1,20.8,17.6,25.3,0,0
2712,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,1,0,수,2분기,1.1,20.8,17.6,25.3,0,0
2713,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,1,0,수,2분기,1.1,20.8,17.6,25.3,0,0


In [66]:
X.loc[X["일"]=="10", ["월급날"]] #잘 들어갔는지 확인
print(X.sum().월급날) #총 월급날

1092


## 금,토 및 공휴일

특히 직장인 같은 경우 금요일 저녁부터 토요일 및 공휴일에 집에서 보내는 시간이 많아져 홈쇼핑을 더 많이 접할 것으로 판단하였다. 이에 금요일, 토요일, 및 공휴일이 홈쇼핑 매출에 긍정적인 영향을 줄 것으로 생각했다.

In [67]:
#https://publicholidays.co.kr/ko/2020-dates/

In [68]:
#2020년 6월에 대해 금요일, 토요일 및 공휴일 날짜 리스트를 뽑았다.
#해당 날짜에 편성된 상품은 '휴일' 변수에서 1의 값을, 다른 날에 편성된 상품은 0의 값을 부여했다.
def holiday(df = X):
    df["휴일"] = 0
    holiday_list=[5,6,7,12,13,14,19,20,21,26,27,28]
    
    for i in holiday_list:
            df.loc[df['일'].astype('float32')==i,'휴일']=1
    return X

In [69]:
X=holiday(X);X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,02:00~04:00,요일,분기,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,0,월,2분기,0.3,20.8,15.8,26.0,0,0,0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,0,월,2분기,0.3,20.8,15.8,26.0,0,0,0
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,0,월,2분기,0.3,20.8,15.8,26.0,0,0,0
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,0,월,2분기,0.3,20.8,15.8,26.0,0,0,0
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,0,월,2분기,0.3,20.8,15.8,26.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,0,수,2분기,1.1,20.8,17.6,25.3,0,0,0
2711,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,0,수,2분기,1.1,20.8,17.6,25.3,0,0,0
2712,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,0,수,2분기,1.1,20.8,17.6,25.3,0,0,0
2713,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,0,수,2분기,1.1,20.8,17.6,25.3,0,0,0


In [70]:
X.loc[(X["월"].astype("float32")==3) & (X["일"].astype("float32")==1),["휴일"]] #잘 들어갔는지 확인
print(X.sum().휴일) #총 휴일

1208


## 상품군 power

각 상품군 별로 '(취급액/판매단가)'의 총합과 평균방송분의 총 합을 구한다 그리고 각 상품군 별로 '(취급액/판매단가)의 총 합'을 '평균방송분'의 총 합으로 나누고 소수점아래 둘째 자리에서 반올림해 분당 판매개수를 구하고 그것을 상품군의 power(얼마나 팔리는지) 라고 정의하였다.

In [71]:
def goods_power(df=X):
    df.loc[:,"상품군power"] = 0
    goods = ['가구','가전','건강기능','농수축','생활용품','속옷','의류','이미용','잡화','주방','침구']
    power = [4.25, 3.39, 22.53, 52.04, 17.25, 24.41, 26.41, 41.32, 26.41, 19.4, 25.11]
    for i in range(len(df)):
        for j in range(len(goods)):
            if df.loc[i,'상품군'] == goods[j]:
                df.loc[i,"상품군power"] = power[j]
    return(df)

In [72]:
X = goods_power(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,요일,분기,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,월,2분기,0.3,20.8,15.8,26.0,0,0,0,26.41
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,월,2분기,0.3,20.8,15.8,26.0,0,0,0,26.41
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,월,2분기,0.3,20.8,15.8,26.0,0,0,0,26.41
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,월,2분기,0.3,20.8,15.8,26.0,0,0,0,24.41
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,월,2분기,0.3,20.8,15.8,26.0,0,0,0,24.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,수,2분기,1.1,20.8,17.6,25.3,0,0,0,24.41
2711,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,수,2분기,1.1,20.8,17.6,25.3,0,0,0,24.41
2712,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,수,2분기,1.1,20.8,17.6,25.3,0,0,0,24.41
2713,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,수,2분기,1.1,20.8,17.6,25.3,0,0,0,26.41


In [73]:
# 통계청의 소비자물가지수를 참고해 6월 해당 수치를 추가해준다.
# 출처: http://www.index.go.kr/potal/main/EachDtlPageDetail.do?idx_cd=1060
X['CPI'] = 104.87

## 소매업태별(홈쇼핑) 판매액지수(불변지수)

In [74]:
# 통계청의 소매업태별 판매액지수 중 홈쇼핑 항목의 불변지수를 참고해 6월 해당 수치를 추가해준다.
# 출처: http://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1K31013&vw_cd=MT_ZTITLE&list_id=JG2&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_ZTITLE
X['RS'] = 139.2

## Prime Time

In [75]:
# https://www.etnews.com/20180718000049?m=1
# 자료 조사결과, 프라임타임이 홈쇼핑 매출액에 중요한 변수가 될 수 있음을 확인
# 특히 ns shop +의 경우 식품(농수축)을 주요 타겟으로 하기에 식품 프라임타임(16~18시)을 별도로 취급.
# 기타 상품군도 고려하여 오전/오후에 각각 프라임타임 지정.

In [76]:
def prime_time(df = X):
    df['PrimeTime'] = 0
    
    for i in range(len(df)):
        if (9 <= df.loc[i,'시각']) and (df.loc[i,'시각'] < 11):
            df.loc[i,'PrimeTime'] = "오전프라임"
        elif df.loc[i,'상품군']=="농수축" and (16<=df.loc[i,'시각']) and (df.loc[i,'시각']< 18):
            df.loc[i,'PrimeTime'] = "식품프라임"            
        elif df.loc[i,'상품군']!="농수축" and (20<=df.loc[i,'시각']) and (df.loc[i,'시각']< 23):
            df.loc[i,'PrimeTime'] = "그외프라임"
        else:
            df.loc[i,'PrimeTime'] = "프라임아님"
    
    return df

## 방송 시작시간을 고려해서 [,)로 시간 처리
## 프라임타임을 세분화해서 오전/식품오후/그외오후/프라임아님 4가지 카테고리로 구분

In [77]:
X = prime_time(X) ; X.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power,CPI,RS,PrimeTime
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.0,1,2020-06-01,...,20.8,15.8,26.0,0,0,0,26.41,104.87,139.2,프라임아님
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.0,2,2020-06-01,...,20.8,15.8,26.0,0,0,0,26.41,104.87,139.2,프라임아님
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.0,3,2020-06-01,...,20.8,15.8,26.0,0,0,0,26.41,104.87,139.2,프라임아님
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.0,1,2020-06-01,...,20.8,15.8,26.0,0,0,0,24.41,104.87,139.2,프라임아님
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.0,2,2020-06-01,...,20.8,15.8,26.0,0,0,0,24.41,104.87,139.2,프라임아님


## 남성, 여성 상품

의류, 속옷, 잡화, 이미용 상품 중 남녀 별로 구분할 수 있는 상품이 많다. 2017년 KISDI에서 실시한 연구에 따르면 TV홈쇼핑 이용자 중 약 70%가 여성이다. 이렇게 남녀별로 홈쇼핑 소비패턴이 다르기 때문에 성별을 고려하였다.

In [78]:
# test 에만 있는 상품을 몇개 추가하였음.
def gender(df=X):
    df["남성상품"]=0
    df["여성상품"]=0
    list_male=["남성","드로즈","푸마 드라이셀라인 뉴 런닝","언더셔츠","카파 런닝","리복 스피드윅 런닝",
               "트렁크","프로스펙스 2020 퍼포먼스 런닝"]
    list_female=["여성","브라","밍크","팬티","치마","레깅스","란쥬","핸드백",
                 "앙상블","엔셀라두스 밴딩팬츠","레이스","엘리자베스아덴","버켓 백","미란다백","모터백",
                 "크로스바디 백","크로스바디백","립스틱","쿠션","젤네일","아이라이너","사첼백","숄더","속눈썹","토트백","호보백",
                 "원피스","틴트","투웨이백","이지탑","크로스백","클러치백","보정런닝", "아이스타일러",
                 "오렐리안 히트모 스킨컬렉션 기모런닝","보정 런닝", "노벨라백", "제니스시계 주얼리세트",
                 "콘체르토 다이아몬드 워치(주얼리3종)", "삭루츠 버켓 백",
                 "무빙스타 다이아몬드 워치", "목걸이", "팔찌", "18K 베르니아 주얼리 세트",
                 "귀걸이", "반지", "펌프스", "링", "사첼", "실크테라피 오리지널 에센스 패키지", "마스카라",
                 "피부약방 더오리진 크림세트","NNF 텐셀팬츠3종","마르엘라로사티","라라쎄 쉬폰 롤업재킷","웨지","페라가모 간치니 쥬얼리 선글라스"]
    for i in ["의류","속옷","잡화","이미용"]:
        for j in list_male:
            df.loc[(df["상품군"]==i)&(df.상품명.str.contains(j)),"남성상품"]=1
        for k in list_female:
            df.loc[(df["상품군"]==i)&(df.상품명.str.contains(k)),"여성상품"]=1
    return df

In [79]:
X = gender(X); X

C:\Users\goran\Anaconda3\envs\tensor\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,최고기온(℃),연예인,월급날,휴일,상품군power,CPI,RS,PrimeTime,남성상품,여성상품
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,26.0,0,0,0,26.41,104.87,139.2,프라임아님,1,0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,26.0,0,0,0,26.41,104.87,139.2,프라임아님,1,0
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,26.0,0,0,0,26.41,104.87,139.2,프라임아님,1,0
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,26.0,0,0,0,24.41,104.87,139.2,프라임아님,0,1
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,26.0,0,0,0,24.41,104.87,139.2,프라임아님,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,25.3,0,0,0,24.41,104.87,139.2,프라임아님,1,0
2711,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,25.3,0,0,0,24.41,104.87,139.2,프라임아님,1,0
2712,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,25.3,0,0,0,24.41,104.87,139.2,프라임아님,1,0
2713,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,25.3,0,0,0,26.41,104.87,139.2,프라임아님,0,1


## 무이자, 일시불 처리

무이자, 일시불과 같은 구매 방식 또한 소비에 영향을 미칠 것이라 판단하여 상품 별 무이자, 일시불을 나타내는 열을 추가했다.

In [80]:
def pay(df=X):
    df["무이자"]=0 
    df["일시불"]=0
    pay_in_inst=["무)", "무이자"] # 무), (무), 무이자 등의 형태로 무이자 상품이 표기
    pay_in_full=["일)", "일시불"] # 일), (일), 일시불 등의 형태로 무이자 상품이 표기
    for i in pay_in_inst:
        df.loc[(df.상품명.str.contains(i,regex=False))&(~df.상품명.str.contains("보험")),"무이자"]=1 # (무)로 표기된 무배당 보험상품 제외
    for j in pay_in_full:
        df.loc[(df.상품명.str.contains(j, regex=False))&(~df.상품명.str.contains("세일")),"일시불"]=1 # (세일)로 표기된 세일 상품 제외
    return df

In [81]:
X = pay(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,월급날,휴일,상품군power,CPI,RS,PrimeTime,남성상품,여성상품,무이자,일시불
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,0,0,26.41,104.87,139.2,프라임아님,1,0,0,0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,0,0,26.41,104.87,139.2,프라임아님,1,0,0,0
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,0,0,26.41,104.87,139.2,프라임아님,1,0,0,0
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,0,0,24.41,104.87,139.2,프라임아님,0,1,0,0
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,0,0,24.41,104.87,139.2,프라임아님,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,0,0,24.41,104.87,139.2,프라임아님,1,0,0,1
2711,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,0,0,24.41,104.87,139.2,프라임아님,1,0,1,0
2712,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,0,0,24.41,104.87,139.2,프라임아님,1,0,0,0
2713,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,0,0,26.41,104.87,139.2,프라임아님,0,1,0,0


## 유명 기업/브랜드명

In [82]:
# train 에 없지만 test 에만 있는 브랜드를 몇개 추가하였음
def brandname(df=X):
    df['유명기업/브랜드']=0
    lst=["LG","SPC삼립","경남제약", "광동", "구찌", "굿프렌드", "농협", "딤채", "디즈니",
         "락앤락","리복","마리끌레르","뱅뱅","버버리", "베지밀","보루네오","본죽","삼성",
         "엘르","종근당건강","쿠쿠",'쿠첸',"코치","푸마","프라다","하림","한샘","버버리",
         "비비고"]
    for i in lst:
        df.loc[df.상품명.str.contains(i),'유명기업/브랜드']=1
    for i in range(len(df)):
        if ("폴로" in df.loc[i,"상품명"]) or ("USPA" in df.loc[i,"상품명"]):
            df.loc[i,"유명기업/브랜드"] = 1
            
    return df

In [83]:
X = brandname(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,휴일,상품군power,CPI,RS,PrimeTime,남성상품,여성상품,무이자,일시불,유명기업/브랜드
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,0,26.41,104.87,139.2,프라임아님,1,0,0,0,0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,0,26.41,104.87,139.2,프라임아님,1,0,0,0,0
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,0,26.41,104.87,139.2,프라임아님,1,0,0,0,0
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,0,24.41,104.87,139.2,프라임아님,0,1,0,0,0
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,0,24.41,104.87,139.2,프라임아님,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,0,24.41,104.87,139.2,프라임아님,1,0,0,1,0
2711,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,0,24.41,104.87,139.2,프라임아님,1,0,1,0,0
2712,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,0,24.41,104.87,139.2,프라임아님,1,0,0,0,0
2713,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,0,26.41,104.87,139.2,프라임아님,0,1,0,0,0


## 타 채널 시청자 수

논문 '홈쇼핑 주문 건수의 확산과 zapping의 효과 분석' 등의 자료를 통해 다른 방송을 시청하다가 중간에 홈쇼핑 채널로 유입되는 경우가 홈쇼핑 방송 시청에 유의미한 영향을 미친다고 생각했다. 따라서 '닐슨코리아'의 2019년 전국 시청자수 상위 20위 방송의 시청자수 데이터를 가공하여 이를 반영하려 했다. 분석 자료에 날짜별 전국 시청자수 상위 20위 방송의 시청자수의 평균을 변수로 추가했다.

In [84]:
# 크롤링에 필요한 패키지 import
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [85]:
# 웹 주소에 들어갈 날짜 부분을 리스트로 정의
date = X['월일']
date = date[~date.duplicated()]
date = list(date)

In [86]:
X['타 채널 시청자 수 평균'] = 0
def rate_add(df=X, date=date):
    # 앞서 설정한 날짜 리스트를 통해 날짜 별 시청률 웹 페이지 호출
    for i in date:
        url = "http://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=1_1&area=00&begin_date=2020"+i
        html = urlopen(url)  
        rate_html = BeautifulSoup(html, "html.parser") 
        rate = []
        # html 형식의 값을 다듬는다
        for j in range(10,20):
            a = rate_html.find_all("td",class_='percent')[j].get_text()
            a=a.replace("\t", "")
            a=a.replace("\r","")
            a=a.replace("\n","")
            a=a.replace(",","")
            a=a.strip()
            rate.append(a)
            
        for k in range(10,20):
            b = rate_html.find_all("td",class_='percent_g')[k].get_text()
            b=b.replace("\t", "")
            b=b.replace("\r","")
            b=b.replace("\n","")
            b=b.replace(",","")
            b=b.strip()
            rate.append(b)
        # 정수형으로 변환한 후 평균값을 도출     
        rate = [int(m) for m in rate]
        rate_mean = sum(rate) / 20
        df.loc[df['월일']==i,'타 채널 시청자 수 평균'] = rate_mean
           
    return  df

In [87]:
X = rate_add(X, date)
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,상품군power,CPI,RS,PrimeTime,남성상품,여성상품,무이자,일시불,유명기업/브랜드,타 채널 시청자 수 평균
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,26.41,104.87,139.2,프라임아님,1,0,0,0,0,1382.15
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,26.41,104.87,139.2,프라임아님,1,0,0,0,0,1382.15
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,26.41,104.87,139.2,프라임아님,1,0,0,0,0,1382.15
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,24.41,104.87,139.2,프라임아님,0,1,0,0,0,1382.15
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,24.41,104.87,139.2,프라임아님,0,1,0,0,0,1382.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,24.41,104.87,139.2,프라임아님,1,0,0,1,0,1242.05
2711,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,24.41,104.87,139.2,프라임아님,1,0,1,0,0,1242.05
2712,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,24.41,104.87,139.2,프라임아님,1,0,0,0,0,1242.05
2713,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,26.41,104.87,139.2,프라임아님,0,1,0,0,0,1242.05


## 가전제품 소분류

가전제품 중 세탁기와 냉장고의 취급액이 에어컨과 노트북의 취급액 보다 두 배 이상 차이났다. 가전제품의 종류에 따라 취급액 차이를 고려하기 위해 "가전제품 소분류" 변수를 생성했다.

In [88]:
#가전제품 소분류는 총 10가지이다. 
X["가전제품"]="가전제품 아님"
list_products=['세탁기', '노트북', '냉장고', 'TV', '건조기', '공기청정기', '무선청소기', '에어컨', '의류관리기']

for j in list_products:
    for i in range(len(X)):
        if X.loc[i,"상품군"]=="가전":
            if j in X.loc[i,"상품명"]:
                X.loc[i,"가전제품"]=j

for i in range(len(X)):
    if X.loc[i,"상품군"]=="가전":
        if "이동식에어컨" in X.loc[i,"상품명"]:
            X.loc[i,"가전제품"]="에어컨"


X.loc[47:150,['상품명',"가전제품"]]

,상품명,가전제품
47,LG 울트라HD TV AI ThinQ(인공지능 씽큐) 75형 75UN7850KNA,TV
48,LG 울트라HD TV AI ThinQ(인공지능 씽큐) 55형 55UN7850KNA,TV
49,LG 울트라HD TV AI ThinQ(인공지능 씽큐) 65형 65UN7850KNA,TV
50,LG 울트라HD TV AI ThinQ(인공지능 씽큐) 75형 75UN7850KNA,TV
51,보루네오 델루나 유로탑 슬라이딩 LED침대 슈퍼싱글,가전제품 아님
...,...,...
146,무이자 삼성 UHDTV 75형 / KU75UT7000FXKR,TV
147,일시불 삼성 UHDTV 75형 KU75UT7000FXKR + 삼성 사운드바HW-T450,TV
148,무이자 삼성 UHDTV 65형 KU65UT7000FXKR,TV
149,일시불 삼성 UHDTV 65형 KU65UT7000FXKR+삼성 사운드 바 / HW-...,TV


## 농수축소분류

In [89]:
# 농수축 상품을 9개의 주요 품목으로 세분화
def AgroFishery(df=X):
    df['농수축소분류']="농수축아님"
    lst_쌀 = ["서산뜸부기쌀","멋진밥상 흥양농협 햅쌀"]
    lst_과일 = ["키위","감귤"]
    lst_김치 = ["김치"]
    lst_어류 = ["제주바다자연산돔","오징어","우럭","가자미","전복","고등어","굴비","홍어","[오세득 프라임 컬렉션] 바로 메로 10팩+특제소스","갈치","꽃게","장어","문어"]
    lst_건어류 = ["쥐포","쥐치포"]
    lst_육류 = ["닭발","뒷고기","곱창","오리","육포","삼계탕","해장국","막창"]
    lst_음료 = ["커피","아침애끌리다","두유"]
    lst_주전부리 = ['옥수수',"오메기떡", "전철우냉면", "다시마국수","만두","갓바위대천김도시락김120봉","갓바위전장김27봉"]
    
    for i in lst_쌀:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="쌀"
    for i in lst_과일:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="과일"
    for i in lst_김치:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="김치"
    for i in lst_어류:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="어류"
    for i in lst_건어류:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="건어류"
    for i in lst_육류:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="육류"
    for i in lst_음료:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="음료"
    for i in lst_주전부리:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="주전부리"


    return df


In [90]:
X = AgroFishery(X)
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,RS,PrimeTime,남성상품,여성상품,무이자,일시불,유명기업/브랜드,타 채널 시청자 수 평균,가전제품,농수축소분류
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,139.2,프라임아님,1,0,0,0,0,1382.15,가전제품 아님,농수축아님
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,139.2,프라임아님,1,0,0,0,0,1382.15,가전제품 아님,농수축아님
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,139.2,프라임아님,1,0,0,0,0,1382.15,가전제품 아님,농수축아님
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,139.2,프라임아님,0,1,0,0,0,1382.15,가전제품 아님,농수축아님
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,139.2,프라임아님,0,1,0,0,0,1382.15,가전제품 아님,농수축아님
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,139.2,프라임아님,1,0,0,1,0,1242.05,가전제품 아님,농수축아님
2711,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,139.2,프라임아님,1,0,1,0,0,1242.05,가전제품 아님,농수축아님
2712,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,139.2,프라임아님,1,0,0,0,0,1242.05,가전제품 아님,농수축아님
2713,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,139.2,프라임아님,0,1,0,0,0,1242.05,가전제품 아님,농수축아님


## 어류 손질 여부

In [91]:
def trimfood(df=X):
    df['어류손질여부']="해당없음"
    for i in range(len(X)):
        if df.loc[i,"농수축소분류"]=="어류":
            if df.상품명.str.contains("손질")[i]:
                df.loc[i,'어류손질여부']=1
            else:
                df.loc[i,'어류손질여부']=0
    return df

In [92]:
X = trimfood(X);X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,날짜,...,PrimeTime,남성상품,여성상품,무이자,일시불,유명기업/브랜드,타 채널 시청자 수 평균,가전제품,농수축소분류,어류손질여부
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,2020-06-01,...,프라임아님,1,0,0,0,0,1382.15,가전제품 아님,농수축아님,해당없음
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,2020-06-01,...,프라임아님,1,0,0,0,0,1382.15,가전제품 아님,농수축아님,해당없음
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,2020-06-01,...,프라임아님,1,0,0,0,0,1382.15,가전제품 아님,농수축아님,해당없음
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,2020-06-01,...,프라임아님,0,1,0,0,0,1382.15,가전제품 아님,농수축아님,해당없음
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,2020-06-01,...,프라임아님,0,1,0,0,0,1382.15,가전제품 아님,농수축아님,해당없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,2020-07-01,...,프라임아님,1,0,0,1,0,1242.05,가전제품 아님,농수축아님,해당없음
2711,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,2020-07-01,...,프라임아님,1,0,1,0,0,1242.05,가전제품 아님,농수축아님,해당없음
2712,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,2020-07-01,...,프라임아님,1,0,0,0,0,1242.05,가전제품 아님,농수축아님,해당없음
2713,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,2020-07-01,...,프라임아님,0,1,0,0,0,1242.05,가전제품 아님,농수축아님,해당없음


In [93]:
#X.drop(columns=['방송일시','마더코드','상품코드','상품명','요일/시간','날짜','년','월','일','시분','상품군power','가전제품'],inplace=True)

In [94]:
X.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '평균방송분', '겹치는수',
       '날짜', '년', '월', '일', '시각', '시분', '월일', '시간열', '정수노출(분)', '시청률평균',
       '요일/시간', '06:00~08:00', '08:00~10:00', '10:00~12:00', '12:00~14:00',
       '14:00~16:00', '16:00~18:00', '18:00~20:00', '20:00~22:00',
       '22:00~00:00', '00:00~02:00', '02:00~04:00', '요일', '분기', '강수량(mm)',
       '평균기온(℃)', '최저기온(℃)', '최고기온(℃)', '연예인', '월급날', '휴일', '상품군power', 'CPI',
       'RS', 'PrimeTime', '남성상품', '여성상품', '무이자', '일시불', '유명기업/브랜드',
       '타 채널 시청자 수 평균', '가전제품', '농수축소분류', '어류손질여부'],
      dtype='object')

## 범주형 자료 dummy 변수화 + 원본을 제거 DROP

모델링을 위해 가공돈 train 데이터와 변수 배열을 같게 만드는 작업이다. 범주형 자료를 dummy 변수화한 후, 바뀐 범주형 자료의 열을 drop했다. 

In [95]:
# 월 변수를 one-hot 변수로 변환
# 기존 월 변수를 drop
def month(df = X):
    df["월_1"] = 0
    df["월_2"] = 0
    df["월_3"] = 0
    df["월_4"] = 0
    df["월_5"] = 0
    df["월_6"] = 1
    df["월_7"] = 0
    df["월_8"] = 0
    df["월_9"] = 0
    df["월_10"] = 0
    df["월_11"] = 0
    df["월_12"] = 0
    
    X.drop(columns=['월'],axis=1,inplace=True)
    
    return X

In [96]:
X=month(X)

In [97]:
# 요일 변수를 one-hot 변수로 변환
# 기존 요일 변수를 drop
def wkday(df = X):
    df["요일_금"] = 0
    df["요일_목"] = 0
    df["요일_수"] = 0
    df["요일_월"] = 0
    df["요일_일"] = 0
    df["요일_토"] = 0
    df["요일_화"] = 0
    
    df.loc[df['요일']=="금",'요일_금']=1
    df.loc[df['요일']=="목",'요일_목']=1
    df.loc[df['요일']=="수",'요일_수']=1
    df.loc[df['요일']=="월",'요일_월']=1
    df.loc[df['요일']=="일",'요일_일']=1
    df.loc[df['요일']=="토",'요일_토']=1
    df.loc[df['요일']=="화",'요일_화']=1
    
    X.drop(columns=['요일'],axis=1,inplace=True)
    
    return X

In [98]:
X=wkday(X)

In [99]:
# 분기 변수를 one-hot 변수로 변환
# 기존 분기 변수를 drop
X['분기_1분기']=0
X['분기_2분기']=1
X['분기_3분기']=0
X['분기_4분기']=0
X.drop(columns=['분기'],axis=1,inplace=True)

In [100]:
# 상품군 변수를 one-hot 변수로 변환
# 기존 상품군 변수를 drop
X['상품군_가구']=0
X['상품군_가전']=0
X['상품군_건강기능']=0
X['상품군_농수축']=0
X['상품군_생활용품']=0
X['상품군_속옷']=0
X['상품군_의류']=0
X['상품군_이미용']=0
X['상품군_잡화']=0
X['상품군_주방']=0
X['상품군_침구']=0
def group(df=X):
    
    for i in range(len(df)):
        if df.loc[i,'상품군']=='가구':
            df.loc[i,'상품군_가구']=1
        elif df.loc[i,'상품군']=='가전':
            df.loc[i,'상품군_가전']=1
        elif df.loc[i,'상품군']=='건강기능':
            df.loc[i,'상품군_건강기능']=1
        elif df.loc[i,'상품군']=='농수축':
            df.loc[i,'상품군_농수축']=1
        elif df.loc[i,'상품군']=='생활용품':
            df.loc[i,'상품군_생활용품']=1
        elif df.loc[i,'상품군']=='속옷':
            df.loc[i,'상품군_속옷']=1
        elif df.loc[i,'상품군']=='의류':
            df.loc[i,'상품군_의류']=1
        elif df.loc[i,'상품군']=='이미용':
            df.loc[i,'상품군_이미용']=1
        elif df.loc[i,'상품군']=='잡화':
            df.loc[i,'상품군_잡화']=1
        elif df.loc[i,'상품군']=='주방':
            df.loc[i,'상품군_주방']=1
        elif df.loc[i,'상품군']=='침구':
            df.loc[i,'상품군_침구']=1

    X.drop('상품군',axis=1,inplace=True)
    
    return df

In [101]:
X=group(X)

In [102]:
# PrimeTime 변수를 one-hot 변수로 변환
# 기존 PrimeTime 변수를 drop
X['PrimeTime_그외프라임']=0
X['PrimeTime_식품프라임']=0
X['PrimeTime_오전프라임']=0
X['PrimeTime_프라임아님']=0
def prime(df=X):
    for i in range(len(df)):
        if df.loc[i,'PrimeTime']=='그외프라임':
            df.loc[i,'PrimeTime_그외프라임']=1
        elif df.loc[i,'PrimeTime']=='식품프라임':
            df.loc[i,'PrimeTime_식품프라임']=1
        elif df.loc[i,'PrimeTime']=='오전프라임':
            df.loc[i,'PrimeTime_오전프라임']=1
        elif df.loc[i,'PrimeTime']=='프라임아님':
            df.loc[i,'PrimeTime_프라임아님']=1
        
    X.drop('PrimeTime',axis=1,inplace=True)
    
    return df

In [103]:
X=prime(X)

In [104]:
# 어류손질여부 변수를 one-hot 변수로 변환
# 기존 어류손질여부 변수를 drop
X['어류손질여부_0']=0
X['어류손질여부_1']=0
X['어류손질여부_해당없음']=0
def fish(df=X):
    for i in range(len(df)):
        if df.loc[i,'어류손질여부']==0:
            df.loc[i,'어류손질여부_0']=1
        elif df.loc[i,'어류손질여부']==1:
            df.loc[i,'어류손질여부_1']=1
        elif df.loc[i,'어류손질여부']=='해당없음':
            df.loc[i,'어류손질여부_해당없음']=1
        
    X.drop('어류손질여부',axis=1,inplace=True)
    
    return df

In [105]:
X=fish(X)

In [106]:
# 가전제품 변수를 one-hot 변수로 변환
# 기존 가전제품 변수를 drop
X['가전제품_TV']=0
X['가전제품_가전제품 아님']=0
X['가전제품_건조기']=0
X['가전제품_공기청정기']=0
X['가전제품_냉장고']=0
X['가전제품_노트북']=0
X['가전제품_무선청소기']=0
X['가전제품_세탁기']=0
X['가전제품_에어컨']=0
X['가전제품_의류관리기']=0
def electric(df=X):
    for i in range(len(df)):
        if df.loc[i,'가전제품']=='TV':
            df.loc[i,'가전제품_TV']=1
        elif df.loc[i,'가전제품']=='가전제품 아님':
            df.loc[i,'가전제품_가전제품 아님']=1
        elif df.loc[i,'가전제품']=='건조기':
            df.loc[i,'가전제품_건조기']=1
        elif df.loc[i,'가전제품']=='공기청정기':
            df.loc[i,'가전제품_공기청정기']=1
        elif df.loc[i,'가전제품']=='냉장고':
            df.loc[i,'가전제품_냉장고']=1
        elif df.loc[i,'가전제품']=='노트북':
            df.loc[i,'가전제품_노트북']=1
        elif df.loc[i,'가전제품']=='무선청소기':
            df.loc[i,'가전제품_무선청소기']=1
        elif df.loc[i,'가전제품']=='세탁기':
            df.loc[i,'가전제품_세탁기']=1
        elif df.loc[i,'가전제품']=='에어컨':
            df.loc[i,'가전제품_에어컨']=1
        elif df.loc[i,'가전제품']=='의류관리기':
            df.loc[i,'가전제품_의류관리기']=1
            

    X.drop('가전제품',axis=1,inplace=True)
    
    return df

In [107]:
X=electric(X)

In [108]:
# 농수축소분류 변수를 one-hot 변수로 변환
# 기존 농수축소분류 변수를 drop
def agrofish_to_int(df = X):
    df['농수축소분류_건어류']=0
    df['농수축소분류_과일']=0
    df['농수축소분류_김치']=0
    df['농수축소분류_분류에없음']=0
    df['농수축소분류_쌀']=0
    df['농수축소분류_어류']=0
    df['농수축소분류_육류']=0
    df['농수축소분류_음료']=0
    df['농수축소분류_장류/조미료']=0
    df['농수축소분류_주전부리']=0
   
    for i in range(len(df)):
        if df.loc[i, "농수축소분류"]=="건어류":
            df.loc[i,'농수축소분류_건어류'] = 1
        elif df.loc[i, "농수축소분류"]=="과일":
            df.loc[i,'농수축소분류_과일'] = 1
        elif df.loc[i, "농수축소분류"]=="김치":
            df.loc[i,'농수축소분류_김치'] = 1
        elif df.loc[i, "농수축소분류"]=="분류에없음":
            df.loc[i,'농수축소분류_분류에없음'] = 1
        elif df.loc[i, "농수축소분류"]=="쌀":
            df.loc[i,'농수축소분류_쌀'] = 1
        elif df.loc[i, "농수축소분류"]=="어류":
            df.loc[i,'농수축소분류_어류'] = 1
        elif df.loc[i, "농수축소분류"]=="육류":
            df.loc[i,'농수축소분류_육류'] = 1
        elif df.loc[i, "농수축소분류"]=="음료":
            df.loc[i,'농수축소분류_음료'] = 1
        elif df.loc[i, "농수축소분류"]=="장류/조미료":
            df.loc[i,'농수축소분류_장류/조미료'] = 1
        elif df.loc[i, "농수축소분류"]=="주전부리":
            df.loc[i,'농수축소분류_주전부리'] = 1
    X.drop(columns=['농수축소분류'],inplace=True)
    return df

In [109]:
X= agrofish_to_int(X)

# 파일 저장

In [110]:
# 예측 데이터를 형성하기 위한 데이터를 저정합니다.
X.to_excel("X_for_predict.xlsx")